In [1]:
# from google.colab import drive
# drive.mount('/content/drive')

In [2]:
import matplotlib.pyplot as plt
import os
import tensorflow as tf
import matplotlib
import keras.backend as K
matplotlib.style.use('ggplot')

2023-02-25 13:29:00.373373: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-25 13:29:03.242083: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/home/pigeon_gcc/anaconda3/envs/iu_ss/lib/
2023-02-25 13:29:03.242335: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/home/pigeon_gcc/anaconda3/envs/iu_ss/lib/
2023-02-25 13:29:03.242362: W tensorflow/c

In [3]:
IMAGE_SHAPE = (400, 300)
TRAINING_DATA_DIR = 'aug/training/'
VALID_DATA_DIR = 'aug/validation/'

In [4]:
datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255
)
train_generator = datagen.flow_from_directory(
    TRAINING_DATA_DIR,
    shuffle=True,
    target_size=IMAGE_SHAPE,
)
valid_generator = datagen.flow_from_directory(
    VALID_DATA_DIR,
    shuffle=False,
    target_size=IMAGE_SHAPE,
)

Found 3557 images belonging to 2 classes.
Found 546 images belonging to 2 classes.


In [5]:
def build_model(num_classes):
    model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(filters=16, kernel_size=(3, 3), activation='relu', 
                           input_shape=(400, 300, 3)),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=2),
    tf.keras.layers.Conv2D(filters=16, kernel_size=(3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=2),
    tf.keras.layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(num_classes, activation='softmax')
    ])
    return model
model = build_model(num_classes=2)

2023-02-25 13:29:06.642480: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-02-25 13:29:06.663480: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-02-25 13:29:06.664160: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-02-25 13:29:06.666245: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorF

In [6]:
def get_f1(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    f1_val = 2*(precision*recall)/(precision+recall+K.epsilon())
    
    return f1_val

In [7]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss=tf.keras.losses.CategoricalCrossentropy(),
    metrics=[get_f1]
)
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 398, 298, 16)      448       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 199, 149, 16)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 197, 147, 16)      2320      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 98, 73, 16)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 96, 71, 32)        4640      
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 48, 35, 32)       0

In [8]:
EPOCHS = 30
BATCH_SIZE = 16
history = model.fit(train_generator,
                    steps_per_epoch=train_generator.samples // BATCH_SIZE // 2,
                    epochs=EPOCHS,
                    validation_data=valid_generator,
                    validation_steps= valid_generator.samples // BATCH_SIZE // 2,
                    verbose=1
                    )

Epoch 1/30


2023-02-25 13:29:13.131881: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:428] Loaded cuDNN version 8100
2023-02-25 13:29:14.427302: I tensorflow/compiler/xla/service/service.cc:173] XLA service 0x7f09e8011930 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-02-25 13:29:14.427332: I tensorflow/compiler/xla/service/service.cc:181]   StreamExecutor device (0): NVIDIA GeForce RTX 2060, Compute Capability 7.5
2023-02-25 13:29:14.431799: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-02-25 13:29:14.567231: I tensorflow/compiler/jit/xla_compilation_cache.cc:477] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


111/111 [==============================] - 22s 152ms/step - loss: 0.7053 - get_f1: 0.5248 - val_loss: 0.6927 - val_get_f1: 0.5147
Epoch 2/30
111/111 [==============================] - 15s 139ms/step - loss: 0.6931 - get_f1: 0.5102 - val_loss: 0.6942 - val_get_f1: 0.4871
Epoch 3/30
111/111 [==============================] - 26s 232ms/step - loss: 0.6937 - get_f1: 0.5302 - val_loss: 0.6946 - val_get_f1: 0.4945
Epoch 4/30
111/111 [==============================] - 48s 429ms/step - loss: 0.6851 - get_f1: 0.5486 - val_loss: 0.6976 - val_get_f1: 0.5276
Epoch 5/30
111/111 [==============================] - 18s 164ms/step - loss: 0.6623 - get_f1: 0.6130 - val_loss: 0.7027 - val_get_f1: 0.5441
Epoch 6/30
111/111 [==============================] - 17s 150ms/step - loss: 0.6610 - get_f1: 0.6221 - val_loss: 0.7003 - val_get_f1: 0.5533
Epoch 7/30
111/111 [==============================] - 48s 434ms/step - loss: 0.5691 - get_f1: 0.6939 - val_loss: 0.6433 - val_get_f1: 0.5956
Epoch 8/30
111/111 [====

In [9]:
# model.save_weights('Artem_Chernitsa')
model.save('Artem_Chernitsa')

INFO:tensorflow:Assets written to: Artem_Chernitsa/assets


INFO:tensorflow:Assets written to: Artem_Chernitsa/assets


In [10]:
# !tar czf Artem_Chernitsa.tar Artem_Chernitsa

In [11]:
# import numpy as np
# image = tf.keras.preprocessing.image.load_img(r"/content/test_barbie/test_barbie.jpg",
#                                               target_size=(640, 480))
# input_arr = tf.keras.preprocessing.image.img_to_array(image)
# input_arr = np.array([input_arr])  # Convert single image to a batch.
# input_arr = input_arr.astype('float32') / 255.  # This is VERY important


# predictions = model.predict(input_arr)

In [12]:
# predictions

In [13]:
# train_generator.class_indices

In [14]:
# predicted_class = np.argmax(predictions, axis=-1)

In [15]:
# predicted_class

In [16]:
# image = tf.keras.preprocessing.image.load_img(r"/content/test_puppy.jpg",
#                                               target_size=(640, 480))
# input_arr = tf.keras.preprocessing.image.img_to_array(image)
# input_arr = np.array([input_arr])  # Convert single image to a batch.
# input_arr = input_arr.astype('float32') / 255.  # This is VERY important


# predictions = model.predict(input_arr)
# predicted_class = np.argmax(predictions, axis=-1)
# print(predictions)
# print(predicted_class)

In [17]:
# model2 = tf.keras.models.load_model('/content/Artem_Chernitsa', compile=False)
# predictions = model2.predict(input_arr)
# predicted_class = np.argmax(predictions, axis=-1)
# print(predictions)
# print(predicted_class)